# Decision Tree
## Model
A decision tree maps input $x \in {R^d}$ to output y using binary decision rules:
 - Each node in the tree has a splitting rule:

$$h\left( x \right) = 1\left\{ {{x_j} > t} \right\}$$

for some dimension $j$ of $x$ and $t \in R$

 - Each leaf node is associated with an output value (outputs can repeat)

Using these transition rules, a path to a leaf node gives the prediction.

## How to fit it

The step to build the basic Decision Tree:

- Step 1: we will start the process with a single leaf node that contains all data.

- Step 2: Loop through the following steps:

    - Calculate the leaf to split that reduces the most inaccuracy.
    - Filter the data: one side is the data that is lower than splitng rule and vice versa.
- Stopping rule duscussed later.


###Build the Regression Tree:
For the  Regression Tree: The squared error is a natural way to define the splitting rule.

For $M$ regions of the space, ${R_1},..,{R_M}$,
the prediction function is

$$f\left( x \right) = \sum\limits_{m = 1}^M {{c_m}1\left\{ {x \in {R_m}} \right\}} $$

So, for a fixed $M$, we need $R_m$ and $c_m$.

Goal: Try to minimize ${\sum\limits_i {\left( {{y_i} - f\left( {{x_i}} \right)} \right)} ^2}$.

1. Find $c_m$ given $R_m$: Simply the average of all $y_i$ for which $x \in R_m$.
2. How do we find regions? Consider splitting region $R$ at value $s$ of $dim j$:
 - define ${R^ - }\left( {j,s} \right) = \left\{ {{x_i} \in R|{x_i}\left( j \right) \leqslant s} \right\}$ and ${R^ + }\left( {j,s} \right) = \left\{ {{x_i} \in R|{x_i}\left( j \right) > s} \right\}$
 - For each dimension $j$, calculate the best splitting point $s$ for that dimension.
 -  Do this for each region (leaf node). Pick the one that reduces the objective most.

###Build the Classification Tree:
For the Classification Tree:  Need some measure of how badly a region classifies data and how much it can improve if it’s split.

K-class problem: For all $ x \in R_m$, let $p_k$ be empirical fraction labeled $k$.

Measures of quality of Rm include:

1. Classification error: $1 - {\max _k}{p_k}$.
2. Gini index: $1 - \sum\limits_k {{p_k}^2} $.
3. Entropy: $ - \sum\limits_k {{p_k}\ln {p_k}} $


## How to use it

In [ ]:
class Node:
	def __init__(self, split_feature,split_value, left_child, right_child,label):
		self.split_feature = split_feature
		self.split_value = split_value
		self.left_child = left_child
		self.right_child = right_child
		self.label = label

class DecisionTree:
	def __init__(self):
		pass

	def fit(self, train):
		train = train.to_numpy()
		self.tree = self.build_tree(train)
	
	def build_tree(self, Train):
		gini_label = self.compute_gini(train[:,0])
		if gini_label < 0.5:
			return None
		best_gini, best_split, best_feature = float("inf"), 0.0, 0.0
		for i in np.random.randint(1, 785, size=100):
			uniques = np.unique(train[:,i])
			for unique in uniques:
				region1 = train[train[:,i] < unique][:,0]
				region2 = train[train[:,i] >= unique][:,0]
				gini1 = self.compute_gini(region1)
				gini2 = self.compute_gini(region2)
				gini = gini1*(len(region1)/len(train)) +  gini1*(len(region2)/len(train))
				if gini < best_gini:
					best_gini, best_split, best_feature = gini, unique, i
		label = np.bincount(train[:,0]).argmax()
		left_node = self.build_tree(train[train[:,best_feature] < best_split])
		right_node = self.build_tree(train[train[:,best_feature] >= best_split])
		return Node(best_feature, best_split, left_node, right_node, label) 

	def compute_gini(self,y):
		dic = {}
		for i in y:
			if i not in  dic:
				dic[i] = 0
			dic[i] += 1
		p = 0
		for i in dic:
			p += (dic[i]/len(y))**2
		gini = 1 - p
		return gini 

	def predict(self, test):
		node = self.tree
		while node.left_child or node.left_child:
			if test[node.split_feature] < node.split_value:
				node = node.left_child
			else:
				node = node.right_child
		return node.label

In [ ]:
import pandas as pd
import numpy as np
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
classifier = DecisionTree()
classifier.fit(df_train)
f = open("outputs/submission2.csv", "w")
f.write("ImageId,Label\n" )
test = df_test
test = test.to_numpy()
for i in range(len(test)):
  a = classifier.predict(test[i])
  f.write(str(i+1) + "," + str(a) +"\n")
f.close()